In [4]:
import ess_premium_analysis
import bbgclient
import datetime
import dfutils
import pandas as pd

In [5]:
#Necessary input items from the database
alpha = "ASH US EQUITY"
CIX_Index = ".ESSASH INDEX"
unaffected_date = "2018-05-01"
tgt_date = "2018-12-04"
exp_close_date = "2019-06-30"

analyst_upside = 86
analyst_downside = 69

#The functions as for the api_host and 
api_host = bbgclient.bbgclient.get_next_available_host()
#Create a dictionary of the peers and weights and the multiples and weights

def create_dict(keys, values):
    return dict(zip(keys, values + [None] * (len(keys) - len(values))))

#Take the peers vs. weights and the multiples vs. weights and create dictionaries
#These two factors must have type dictionary for input
#peersVSweights = create_dict(peers_names, peers_weights)
peersVSweights = {'CBM US EQUITY':0.28, 'SXT US EQUITY': 0.28, 'IFF US EQUITY': 0.28, 'EMN US EQUITY': 0.16}
#multiplesVSweights = create_dict(multiples_names, multiples_weights)
multiplesVSweights = {'EV/Sales': 0, 'EV/EBITDA': 1, 'P/E': 0, 'DVD Yield': 0, 'FCF Yield': 0}

as_of_dt = datetime.datetime.today()
unaff_dt = datetime.datetime.strptime(unaffected_date, '%Y-%m-%d')
tgt_dt = datetime.datetime.strptime(tgt_date, '%Y-%m-%d')

In [5]:
#adj_df = pd.DataFrame(columns = ['CUR_MKT_CAP','EQY_SH_OUT','BEST_EBITDA','BEST_SALES',
            #'BEST_EPS','DIVIDEND_INDICATED_YIELD','BEST_OPP','BEST_NET_INCOME','BEST_CAPEX','CUR_EV_COMPONENT'])
#adj_df.loc[0] = [300,100,100,100,3,100,100,10,10,10]
#adj_df

In [6]:
#Function that takes input and runs the ess_premium_analysis.py document
#The output of this function is the rsults of each model as dataframes
def run_ess_premium_analysis(alpha_ticker, unaffectedDt, tgtDate, as_of_dt, analyst_upside, peers2weight, metric2weight, api_host, f_period = '1BF'):
    slicer = dfutils.df_slicer()
    start_date = slicer.prev_n_business_days(120, unaffectedDt) #lookback is 120 days (6 months)
    metrics = {k: v for k, v in metric2weight.items() if v != 0}
    peers_list = list(peers2weight.keys())
    metric_list = list(metrics.keys())

    calib_data = ess_premium_analysis.calibration_data(alpha_ticker, peers2weight, start_date, unaffectedDt, metric_list, api_host, f_period)

    OLS_results = ess_premium_analysis.premium_analysis_df_OLS(alpha_ticker, peers_list, calib_data, analyst_upside, as_of_dt, tgtDate, metric_list, metric2weight, api_host)
    premium_analysis_results = ess_premium_analysis.premium_analysis_df(alpha_ticker, peers_list, as_of_dt, tgtDate, analyst_upside, metric_list, metric2weight, calib_data['metric2rel'], api_host)

    return(premium_analysis_results, OLS_results)

#model1, model2 = run_ess_premium_analysis(alpha, unaff_dt, tgt_dt, as_of_dt, analyst_upside, peersVSweights, multiplesVSweights, api_host)
#model1

In [7]:
#model2

In [8]:
def final_df(alpha_ticker, cix_index, unaffectedDt, expected_close, tgtDate, analyst_upside, analyst_downside, peers2weight, metric2weight, api_host, f_period = "1BF"):
    slicer = dfutils.df_slicer()
    as_of_dt = datetime.datetime.today()
    unaff_dt = datetime.datetime.strptime(unaffectedDt, '%Y-%m-%d')
    tgt_dt = datetime.datetime.strptime(tgtDate, '%Y-%m-%d')
    exp_close_dt = datetime.datetime.strptime(expected_close, '%Y-%m-%d')
    
    df = pd.Series()
    df["Alpha Ticker"] = alpha_ticker
    df["Unaffected Date"] = unaff_dt.strftime("%Y-%m-%d")
    df["CIX"] = cix_index
    df["Expected Close Date"] = exp_close_dt.strftime('%Y-%m-%d')
    
    df["Alpha Downside"] = analyst_downside
    df["Alpha Upside"] = analyst_upside
    
    df["CIX at Target Date"] = bbgclient.bbgclient.get_timeseries(cix_index,'PX_LAST', slicer.prev_n_business_days(100,tgt_dt).strftime('%Y%m%d'), tgt_dt.strftime('%Y%m%d'), api_host=api_host).iloc[-1]
    df["Alpha at Target Date"] = bbgclient.bbgclient.get_timeseries(alpha_ticker,'PX_LAST', slicer.prev_n_business_days(100,tgt_dt).strftime('%Y%m%d'), tgt_dt.strftime('%Y%m%d'), api_host=api_host).iloc[-1]
    df["Alpha Last Price"] = bbgclient.bbgclient.get_timeseries(alpha_ticker,'PX_LAST', slicer.prev_n_business_days(100,as_of_dt).strftime('%Y%m%d'), as_of_dt.strftime('%Y%m%d'), api_host=api_host).iloc[-1]
    df["CIX Last Price"] = bbgclient.bbgclient.get_timeseries(cix_index,'PX_LAST', slicer.prev_n_business_days(100,as_of_dt).strftime('%Y%m%d'), as_of_dt.strftime('%Y%m%d'),api_host=api_host).iloc[-1]
    
    df["CIX Upside"] = (df["Alpha Upside"] - df["Alpha at Target Date"]) + df["CIX at Target Date"]
    df["CIX Downside"] = df["CIX at Target Date"] - (df["Alpha at Target Date"] - df["Alpha Downside"])
    df["Down Price (CIX)"] = df["Alpha Last Price"] - (df["CIX Last Price"] - df["CIX Downside"])
    df["Up Price (CIX)"] = df["Alpha Last Price"] + (df["CIX Upside"] - df["CIX Last Price"])
    
    model1, model2 = run_ess_premium_analysis(alpha_ticker, unaff_dt, tgt_dt, as_of_dt, analyst_upside, peers2weight, metric2weight, api_host, f_period = "1BF")
    
    df["Down Price (WIC)"] = model1["Alpha Downside (Adj,weighted)"].sum()
    df["Down Price (Regression)"] = model2["Alpha Downside (Adj,weighted)"].sum()
    df["Up Price (WIC)"] = model1["Alpha Upside (Adj,weighted)"].sum()
    df["Up Price (Regression)"] = model2["Alpha Upside (Adj,weighted)"].sum()
    
    df["Embedded Premium($)"] = df["Alpha Last Price"] - ((df["Down Price (WIC)"] + df["Up Price (WIC)"])/2.0)
    df["Embedded Premium(%)"] = 100.0*(df["Embedded Premium($)"]/df["Alpha Last Price"])
    
    df["Probability (CIX)"] = 100.0*((df["CIX Last Price"] - df["CIX Downside"])/(df["CIX Upside"] - df["CIX Downside"]))
    df["Probability (Alpha)"] = 100.0*((df["Alpha Last Price"] - df["Alpha Downside"])/(df["Alpha Upside"] - df["Alpha Downside"]))
    if pd.isna(df['Probability (CIX)']):
        df['Probability(%)'] = df['Probability (Alpha)']
    else:
        df['Probability(%)'] = df['Probability (CIX)']
    #df['Probability(%)'] = [prob_cix if not pd.isnull(prob_cix) else prob_alpha for (prob_cix, prob_alpha) in zip(df["Probability (CIX)"], df["Probability (Alpha)"])]
    
    df["Return/Risk (CIX)"] = (df["CIX Upside"] - df["CIX Last Price"])/(df["CIX Last Price"] - df["CIX Downside"])
    df["Return/Risk (Alpha)"] = (df["Alpha Upside"] - df["Alpha Last Price"])/(df["Alpha Last Price"] - df["Alpha Downside"])
    if not pd.isna(df['Return/Risk (CIX)']):
        df['Return/Risk'] = df['Return/Risk (CIX)']
    else:
        df['Return/Risk'] = df['Probability (Alpha)']
    #df["Return/Risk"] = [radj_cix if not pd.isnull(radj_cix) else radj_alpha for (radj_cix, radj_alpha) in zip(df["Return/Risk (CIX)"], df["Return/Risk (Alpha)"])]
    
    df["Gross Return (CIX)"] = 100.0*((df["CIX Upside"] - df["CIX Last Price"])/df["Alpha Last Price"])
    df["Gross Return (Alpha)"] = 100.0*((df["Alpha Upside"] - df["Alpha Last Price"])/df["Alpha Last Price"])
    if not pd.isna(df['Gross Return (CIX)']):
        df['Gross Return(%)'] = df['Gross Return (CIX)']
        gross = df['Gross Return(%)']
    else:
        df['Gross Return(%)'] = df['Gross Return (Alpha)']
        gross = df['Gross Return(%)']
    #df['Gross Return(%)'] = [gross_ret_cix if not pd.isnull(gross_ret_cix) else gross_ret_alpha for (gross_ret_cix, gross_ret_alpha) in zip(df["Gross Return (CIX)"], df["Gross Return (Alpha)"])]
       
        
    #d1 = datetime.datetime.strptime(expected_close, '%Y-%m-%d')
    #d1 = datetime.datetime.strptime(as_of_dt, '%Y-%m-%d')
    days = abs((exp_close_dt - as_of_dt).days)
    df["Days to Close"] = [None if pd.isnull(days) else days]
    if (pd.isna(df['Gross Return(%)']) or pd.isna(df["Days to Close"]) or days <= 7):
        df['Ann. Return(%)'] = None
    else:
        df['Ann. Return(%)'] = 100.0*((1+gross/100)**(365.0/days)-1.0)
    #df['Ann. Return(%)'] = [None if (pd.isnull(ret) or pd.isnull(days) or days <= 7) else 100.0*((1+ret/100)**(365.0/days)-1.0) for (ret, days) in zip(df['Gross Return(%)'] , df["Days to Close"])]                
    
    
    float_cols = ["Alpha Upside",
                "Alpha Downside",
                "Alpha Last Price",
                'Up Price (CIX)',
                'Down Price (CIX)',
                'CIX Last Price',
                'Return/Risk',
                'Probability(%)',
                'Gross Return(%)',
                'Ann. Return(%)',
                'Up Price (WIC)',
                'Down Price (WIC)',
                'Up Price (Regression)',
                'Down Price (Regression)',
                'Embedded Premium($)',
                'Embedded Premium(%)']
    df[float_cols] = df[float_cols].astype(float)
    
    cols2show = ['Alpha Ticker',
                 'Unaffected Date',
                 'Expected Close Date',
                 'CIX',
                 'Alpha Last Price',
                 'CIX Last Price',
                 'Alpha Downside',
                 'Down Price (CIX)',
                 'Down Price (WIC)',
                 'Down Price (Regression)',
                 'Alpha Upside',
                 'Up Price (CIX)',
                 'Up Price (WIC)',
                 'Up Price (Regression)',
                 'Embedded Premium($)',
                 'Embedded Premium(%)',
                 'Probability(%)',
                 'Return/Risk',
                 'Gross Return(%)',
                 'Ann. Return(%)']
    
    return(df[cols2show])
    

In [23]:
#adj_df = None
#if adj_df is None:
    #x = final_df(alpha, CIX_Index, unaffected_date, exp_close_date, tgt_date, analyst_upside, analyst_downside, peersVSweights, multiplesVSweights, api_host, adjustments_df = None, f_period = "1BF")
#else:
   # x = final_df(alpha, CIX_Index, unaffected_date, exp_close_date, tgt_date, analyst_upside, analyst_downside, peersVSweights, multiplesVSweights, api_host, adjustments_df = adj_df, f_period = "1BF")

In [9]:
final_df(alpha, CIX_Index, unaffected_date, exp_close_date, tgt_date, analyst_upside, analyst_downside, peersVSweights, multiplesVSweights, api_host, f_period = '1BF' )

           Date     ASH     CBM     SXT     IFF    EMN
0    2017-11-06   9.696   7.555  13.579  15.857  8.620
1    2017-11-07   9.449   8.845  13.465  16.175  8.633
2    2017-11-08   9.357   8.289  13.532  15.862  8.629
3    2017-11-09   9.342   8.376  13.492  15.753  8.588
4    2017-11-10   9.428   8.423  13.468  15.752  8.617
5    2017-11-13   9.540   8.337  13.598  16.030  8.657
6    2017-11-14   9.498   8.260  13.596  16.115  8.579
7    2017-11-15   9.559   8.615  13.428  15.851  8.533
8    2017-11-16   9.889   8.797  13.571  15.908  8.568
9    2017-11-17   9.889   8.759  13.599  15.919  8.540
10   2017-11-20   9.803   8.758  13.641  16.028  8.557
11   2017-11-21   9.811   9.005  13.771  16.194  8.614
12   2017-11-22   9.797   8.910  13.643  16.114  8.602
13   2017-11-24   9.857   8.872  13.634  16.244  8.615
14   2017-11-27   9.812   8.815  13.652  16.152  8.596
15   2017-11-28   9.880   8.882  13.765  16.433  8.644
16   2017-11-29  10.116   9.007  13.885  16.438  8.624
17   2017-

Alpha Ticker               ASH US EQUITY
Unaffected Date               2018-05-01
Expected Close Date           2019-06-30
CIX                        .ESSASH INDEX
Alpha Last Price                   75.51
CIX Last Price                    23.758
Alpha Downside                        69
Down Price (CIX)                   66.84
Down Price (WIC)                 53.6156
Down Price (Regression)          69.8722
Alpha Upside                          86
Up Price (CIX)                     83.84
Up Price (WIC)                   82.6933
Up Price (Regression)            85.1255
Embedded Premium($)              7.35556
Embedded Premium(%)              9.74117
Probability(%)                        51
Return/Risk                     0.960784
Gross Return(%)                  11.0317
Ann. Return(%)                   21.0432
dtype: object

In [12]:
as_of_dt = datetime.datetime.today()
unaff_dt = datetime.datetime.strptime(unaffected_date, '%Y-%m-%d')
tgt_dt = datetime.datetime.strptime(tgt_date, '%Y-%m-%d')
exp_close_dt = datetime.datetime.strptime(exp_close_date, '%Y-%m-%d')
alpha_ticker = alpha
cix_index = CIX_Index
peers2weight = peersVSweights
metric2weight = multiplesVSweights
f_period = "1BF"
slicer = dfutils.df_slicer()
    
df = pd.Series()
df["Alpha Ticker"] = alpha_ticker
df["Unaffected Date"] = unaff_dt.strftime("%Y-%m-%d")
df["CIX"] = cix_index
df["Expected Close Date"] = exp_close_dt.strftime('%Y-%m-%d')

df["Alpha Downside"] = analyst_downside
df["Alpha Upside"] = analyst_upside

df["CIX at Target Date"] = bbgclient.bbgclient.get_timeseries(cix_index,'PX_LAST', slicer.prev_n_business_days(100,tgt_dt).strftime('%Y%m%d'), tgt_dt.strftime('%Y%m%d'), api_host=api_host).iloc[0]
df["Alpha at Target Date"] = bbgclient.bbgclient.get_timeseries(alpha_ticker,'PX_LAST', slicer.prev_n_business_days(100,tgt_dt).strftime('%Y%m%d'), tgt_dt.strftime('%Y%m%d'), api_host=api_host).iloc[0]
df["Alpha Last Price"] =  bbgclient.bbgclient.get_timeseries(alpha_ticker,'PX_LAST', slicer.prev_n_business_days(100,as_of_dt).strftime('%Y%m%d'), as_of_dt.strftime('%Y%m%d'), api_host=api_host).iloc[0]
df["CIX Last Price"] = bbgclient.bbgclient.get_timeseries(cix_index,'PX_LAST', slicer.prev_n_business_days(100,as_of_dt).strftime('%Y%m%d'), as_of_dt.strftime('%Y%m%d'),api_host=api_host).iloc[0]

df["CIX Upside"] = (df["Alpha Upside"] - df["Alpha at Target Date"]) + df["CIX at Target Date"]
df["CIX Downside"] = df["CIX at Target Date"] - (df["Alpha at Target Date"] - df["Alpha Downside"])
df["Down Price (CIX)"] = df["Alpha Last Price"] - (df["CIX Last Price"] - df["CIX Downside"])
df["Up Price (CIX)"] = df["Alpha Last Price"] + (df["CIX Upside"] - df["CIX Last Price"])

model1, model2 = run_ess_premium_analysis(alpha_ticker, unaff_dt, tgt_dt, as_of_dt, analyst_upside, peers2weight, metric2weight, api_host, f_period)
    
df["Down Price (WIC)"] = model1.iloc[0]["Alpha Downside (Adj,weighted)"]
df["Down Price (Regression)"] = model2.iloc[0]["Alpha Downside (Adj,weighted)"]
df["Up Price (WIC)"] = model1.iloc[0]["Alpha Upside (Adj,weighted)"]
df["Up Price (Regression)"] = model2.iloc[0]["Alpha Upside (Adj,weighted)"]

print(df)


Alpha Ticker            LOW US EQUITY
Unaffected Date            2018-01-11
CIX                     .ESSLOW INDEX
Expected Close Date        2019-12-31
Alpha Downside                     76
Alpha Upside                      114
CIX at Target Date             0.0621
Alpha at Target Date            99.46
Alpha Last Price                98.59
CIX Last Price                -1.3457
dtype: object


In [41]:
df["Embedded Premium($)"] = df["Alpha Last Price"] - ((df["Down Price (WIC)"] + df["Up Price (WIC)"])/2.0)
df["Embedded Premium(%)"] = 100.0*(df["Embedded Premium($)"]/df["Alpha Last Price"])

df["Probability (CIX)"] = 100.0*((df["CIX Last Price"] - df["CIX Downside"])/(df["CIX Upside"] - df["CIX Downside"]))
df["Probability (Alpha)"] = 100.0*((df["Alpha Last Price"] - df["Alpha Downside"])/(df["Alpha Upside"] - df["Alpha Downside"]))
if not pd.isna(df['Probability (CIX)']):
    df['Probability(%)'] = df['Probability (CIX)']
else:
    df['Probability(%)'] = df['Probability (Alpha)']
#df['Probability(%)'] = [prob_cix if not pd.isnull(prob_cix) else prob_alpha for (prob_cix, prob_alpha) in zip(df["Probability (CIX)"], df["Probability (Alpha)"])]

df["Return/Risk (CIX)"] = (df["CIX Upside"] - df["CIX Last Price"])/(df["CIX Last Price"] - df["CIX Downside"])
df["Return/Risk (Alpha)"] = (df["Alpha Upside"] - df["Alpha Last Price"])/(df["Alpha Last Price"] - df["Alpha Downside"])
if not pd.isna(df['Return/Risk (CIX)']):
    df['Return/Risk'] = df['Return/Risk (CIX)']
else:
    df['Return/Risk'] = df['Probability (Alpha)']
#df["Return/Risk"] = [radj_cix if not pd.isnull(radj_cix) else radj_alpha for (radj_cix, radj_alpha) in zip(df["Return/Risk (CIX)"], df["Return/Risk (Alpha)"])]
    
df["Gross Return (CIX)"] = 100.0*((df["CIX Upside"] - df["CIX Last Price"])/df["Alpha Last Price"])
df["Gross Return (Alpha)"] = 100.0*((df["Alpha Upside"] - df["Alpha Last Price"])/df["Alpha Last Price"])
if not pd.isna(df['Gross Return (CIX)']):
    df['Gross Return(%)'] = df['Gross Return (CIX)']
    gross = df['Gross Return(%)']
else:
    df['Gross Return(%)'] = df['Gross Return (Alpha)']
    gross = df['Gross Return(%)']
#df['Gross Return(%)'] = [gross_ret_cix if not pd.isnull(gross_ret_cix) else gross_ret_alpha for (gross_ret_cix, gross_ret_alpha) in zip(df["Gross Return (CIX)"], df["Gross Return (Alpha)"])]
       
        
#d1 = datetime.datetime.strptime(expected_close, '%Y-%m-%d')
#d1 = datetime.datetime.strptime(as_of_dt, '%Y-%m-%d')
days = abs((exp_close_dt - as_of_dt).days)
df["Days to Close"] = [None if pd.isnull(days) else days]
if (pd.isna(df['Gross Return(%)']) or pd.isna(df["Days to Close"]) or days <= 7):
    df['Ann. Return(%)'] = None
else:
    df['Ann. Return(%)'] = 100.0*((1+gross/100)**(365.0/days)-1.0)
#df['Ann. Return(%)'] = [None if (pd.isnull(ret) or pd.isnull(days) or days <= 7) else 100.0*((1+ret/100)**(365.0/days)-1.0) for (ret, days) in zip(df['Gross Return(%)'] , df["Days to Close"])]                
    
print(df)

Alpha Ticker               PZZA US EQUITY
Unaffected Date                2018-07-11
CIX                        .ESSPZZA INDEX
Expected Close Date            2019-06-30
Alpha Downside                         33
Alpha Upside                         62.5
CIX at Target Date                -7.1765
Alpha at Target Date                38.94
Alpha Last Price                    54.15
CIX Last Price                       5.33
CIX Upside                        16.3835
CIX Downside                     -13.1165
Down Price (CIX)                  35.7035
Up Price (CIX)                    65.2035
Down Price (WIC)                  31.1809
Down Price (Regression)           35.6249
Up Price (WIC)                    64.7119
Up Price (Regression)             65.5158
Embedded Premium($)               6.20361
Embedded Premium(%)               11.4563
Probability (CIX)                 62.5305
Probability (Alpha)               71.6949
Probability(%)                    62.5305
Return/Risk (CIX)                0